In [1]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import pandas as pd
import re  # 숫자추출을 위한 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep


In [2]:
def scrape_hs_legend_data():
    # 엑셀 만들기
    wb = openpyxl.Workbook()

    # 워크시트 만들기
    ws = wb.active
    ws.title = '하스스톤 전설등급 유저수'

    # 데이터 추가하기
    ws.append(['시즌', '년도', '월', '서버', '유저수'])

    # 엑셀 저장하기
    wb.save('./hs_legend.xlsx')

    # 서버, 시즌 변수 정의
    servers = ['AP', 'US', 'EU']  # 아시아태평양,북미,유럽
    start_year = 2022
    start_month = 10  # 시작 시즌 년 월
    num_seasons = 14  # 121시즌까지의 총 개수

    # 엑셀 열기
    wb = openpyxl.load_workbook('./hs_legend.xlsx')
    ws = wb.active

    # 데이터를 저장할 리스트 생성
    data = []

    # ChromeOptions 설정
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    for season in range(108, 108 + num_seasons):
        for server in servers:
            # ChromeDriverManager를 사용하여 Chrome 드라이버를 초기화합니다.
            driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

            # URL 열기
            url = f"https://hearthstone.blizzard.com/ko-kr/community/leaderboards?region={server}&seasonId={season}"
            driver.get(url)

            # metadata 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
            try:
                element_present = EC.presence_of_element_located((By.CLASS_NAME, 'metadata'))
                WebDriverWait(driver, 10).until(element_present)
            except Exception as e:
                print(f"엘리먼트 대기 중 예외 발생: {e}")
                driver.quit()
                continue

            # BeautifulSoup을 사용하여 HTML 파싱
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # metadata 엘리먼트 선택
            metadata = soup.select_one('.metadata')
            print('metadata', metadata)

            # 만약 metadata가 존재한다면 숫자를 추출하고 워크시트와 데이터 리스트에 추가
            if metadata:
                num_text = metadata.text
                num_match = re.search(r'\d+', num_text)
                num = num_match.group()
                # '년도'와 '월' 데이터 추가
                ws.append([season, start_year, start_month, server, num])

                # 데이터 리스트에도 추가
                data.append([season, start_year, start_month, server, num])

            # Chrome 드라이버 종료
            driver.quit()

        # 다음 시즌의 시작년도와 시작월 설정
        start_month += 1
        if start_month > 12:
            start_month = 1
            start_year += 1

    # .xlsx 형식으로 엑셀 저장
    wb.save('./hs_legend.xlsx')

    # 데이터 리스트 반환
    return data

In [3]:
# 함수 호출하여 데이터 수집
scraped_data = scrape_hs_legend_data()

# 데이터를 포함한 리스트에서 DataFrame을 생성
df = pd.DataFrame(scraped_data, columns=['Season', 'Year', 'Month', 'Server', 'Number'])

# DataFrame을 UTF-8 인코딩으로 CSV 파일로 저장
df.to_csv('./hs_legend.csv', encoding='utf-8', index=False)

metadata <div class="metadata"><h2>상위 7810위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <div class="metadata"><h2>상위 10407위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <div class="metadata"><h2>상위 16533위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <div class="metadata"><h2>상위 6745위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <div class="metadata"><h2>상위 10260위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <div class="metadata"><h2>상위 15820위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <div class="metadata"><h2>상위 11005위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <div class="metadata"><h2>상위 18327위 – 정규</h2><p>이 플레이어들은 탁월한 덱 설계 능력과 전략, 순간적인 판단력을 바탕으로 엄청난 업적을 이뤄냈습니다!</p></div>
metadata <